# `CLOUD.jl` - 2D linear advection example

In [36]:
using CLOUD, OrdinaryDiffEq, TimerOutputs

Define the physical problem

In [37]:
θ = π/4 # wave angle
a = sqrt(2) # wave speed
A = 1.0  # amplitude
L = 1.0  # domain length
k = (2*π/L, 2*π/L)  # wave number
T = 1.0/(a*max(abs(cos(θ)),abs(cos(θ)))) # end time for one period

conservation_law = LinearAdvectionEquation((a*cos(θ),a*sin(θ)))
initial_data = InitialDataSine(A,k)
exact_solution = ExactSolution(conservation_law,initial_data);

Set discretization parameters

In [38]:
M = 4
p = 4
p_map = 4
form = WeakConservationForm()
strategy = PhysicalOperator()

ode_algorithm = CarpenterKennedy2N54()
dt = 0.1*(L/M)/(a*(2*p+1));
write_interval = floor(Int, T/(dt*10));

Set up a `DGSEM` scheme with LG quadrature on quadrilaterals of degree `p` with `M` edges in each direction

In [39]:
reference_approximation =ReferenceApproximation(
    DGSEM(p), Quad(),
    volume_quadrature_rule=LGQuadrature(),
    facet_quadrature_rule=LGQuadrature(),
    mapping_degree=p_map, N_plot=50)

mesh = warp_mesh(uniform_periodic_mesh(
    reference_approximation.reference_element, 
    ((0.0,L),(0.0,L)), (M,M)), 
    reference_approximation.reference_element, 0.2)

spatial_discretization = SpatialDiscretization(mesh, 
    reference_approximation)

results_path = save_project(conservation_law,
     spatial_discretization, initial_data, form, (0.0, T), ReferenceOperator(),
     "results/advection_2d_dgsem/", overwrite=true, clear=true)

"results/advection_2d_dgsem/"

Run the solver

In [44]:
ode_problem_1 = semidiscretize(conservation_law,
    spatial_discretization,
    initial_data, 
    form,
    (0.0, T),
    strategy);

save_solution(ode_problem_1.u0, 0.0, results_path, 0)

reset_timer!(timer)
sol = solve(ode_problem_1, ode_algorithm, adaptive=false, dt=dt,
    save_everystep=false, callback=save_callback(results_path, write_interval))
print_timer(timer)

save_solution(last(sol.u), last(sol.t), results_path, "final")

 ───────────────────────────────────────────────────────────────────────────────
                                       Time                    Allocations      
                              ───────────────────────   ────────────────────────
       Tot / % measured:           515ms /  97.6%            493MiB /  99.8%    

 Section              ncalls     time    %tot     avg     alloc    %tot      avg
 ───────────────────────────────────────────────────────────────────────────────
 rhs!                  2.55k    503ms  100.0%   197μs    492MiB  100.0%   197KiB
   apply operators     40.8k    108ms   21.4%  2.64μs   55.4MiB   11.3%  1.39KiB
     volume terms      40.8k   57.8ms   11.5%  1.42μs   22.4MiB    4.6%     576B
     facet terms       40.8k   25.0ms    5.0%   613ns   10.0MiB    2.0%     256B
   gather ext state    40.8k   99.1ms   19.7%  2.43μs    137MiB   27.9%  3.44KiB
   eval num flux       40.8k   80.9ms   16.1%  1.98μs    108MiB   21.9%  2.70KiB
   eval flux           40.8

Calculate the integral L<sup>2</sup> error norm and conservation/energy error

In [41]:
error_analysis = ErrorAnalysis(results_path, conservation_law, 
    spatial_discretization)
conservation_analysis = PrimaryConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)
energy_analysis = EnergyConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)

println("L2 error:")
println(analyze(error_analysis, last(sol.u), initial_data, T)...)
println("Conservation (initial/final/diff):")
println(analyze(conservation_analysis)...)
println("Energy (initial/final/diff):")
println(analyze(energy_analysis)...)

L2 error:
0.005749119046586738
Conservation (initial/final/diff):
[8.023589626732219e-8][8.023589458117097e-8][-1.6861512186494565e-15]
Energy (initial/final/diff):
[0.12499874137052301][0.12439699898414924][-0.0006017423863737686]


Write output to VTK file

In [42]:
postprocess_vtk(spatial_discretization, string(results_path,"solution.vtu"), last(sol.u));